# **Data Collection Notebook**

## Objectives

* Fetch data from Kaggle and prepare it for further processes.

## Inputs
* [https://www.kaggle.com/datasets/jakeshbohaju/brain-tumor](https://www.kaggle.com/datasets/jakeshbohaju/brain-tumor)
*   Kaggle JSON file - the authentication token. 

## Outputs

* Generate Dataset: 
    * input/
    * └── Brain Tumor/ (Image files)
    * ├── Brain Tumor.csv
    * └── bt_dataset_t3.csv

## Additional Comments | Insights | Conclusions

Brain Tumor Data Set
- This dataset includes the Brain MRI image files and two csv files.

- The csv files contain brain tumor feature dataset including five first-order features and eight texture features with the target level (in the column Class).

    - First Order Features
        - Mean
        - Variance
        - Standard Deviation
        - Skewness
        - Kurtosis

    - Second Order Features
        - Contrast
        - Energy
        - ASM (Angular second moment)
        - Entropy
        - Homogeneity
        - Dissimilarity
        - Correlation
        - Coarseness 

- Image column defines image name and Class column defines either the image has tumor or not (1 = Tumor, 0 = Non-Tumor). These two feature are the ones we will take into consideration while classifying the images.



---

## Import packages

In [1]:
%pip install -r ../requirements.txt

  Using cached streamlit-0.85.0-py2.py3-none-any.whl (7.9 MB)
  Using cached altair-4.2.2-py3-none-any.whl (813 kB)
  Using cached astor-0.8.1-py2.py3-none-any.whl (27 kB)
  Using cached attrs-23.1.0-py3-none-any.whl (61 kB)
  Using cached base58-2.1.1-py3-none-any.whl (5.6 kB)
  Obtaining dependency information for blinker from https://files.pythonhosted.org/packages/fa/2a/7f3714cbc6356a0efec525ce7a0613d581072ed6eb53eb7b9754f33db807/blinker-1.7.0-py3-none-any.whl.metadata
  Using cached blinker-1.7.0-py3-none-any.whl.metadata (1.9 kB)
  Obtaining dependency information for cachetools>=4.0 from https://files.pythonhosted.org/packages/a2/91/2d843adb9fbd911e0da45fbf6f18ca89d07a087c3daa23e955584f90ebf4/cachetools-5.3.2-py3-none-any.whl.metadata
  Using cached cachetools-5.3.2-py3-none-any.whl.metadata (5.2 kB)
  Using cached click-7.1.2-py2.py3-none-any.whl (82 kB)
  Obtaining dependency information for numpy from https://files.pythonhosted.org/packages/98/5d/5738903efe0ecb73e51eb44feafba

# Change working directory

* Because of the Jupyter notebooks being in a subfolder, we need to change the directory for the code's execution

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [32]:
import os
current_dir = os.getcwd()
current_dir

'/home/tom/codeinstitute/brain-tumor-detect'

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [3]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [4]:
current_dir = os.getcwd()
current_dir

'/home/tom/codeinstitute/brain-tumor-detect'

## Setup Kaggle

### Install Kaggle

In [5]:
%pip install kaggle==1.5.12

  Using cached kaggle-1.5.12-py3-none-any.whl
  Obtaining dependency information for tqdm from https://files.pythonhosted.org/packages/00/e5/f12a80907d0884e6dff9c16d0c0114d81b8cd07dc3ae54c5e962cc83037e/tqdm-4.66.1-py3-none-any.whl.metadata
  Using cached tqdm-4.66.1-py3-none-any.whl.metadata (57 kB)
  Using cached python_slugify-8.0.1-py2.py3-none-any.whl (9.7 kB)
  Using cached text_unidecode-1.3-py2.py3-none-any.whl (78 kB)
Using cached tqdm-4.66.1-py3-none-any.whl (78 kB)

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


setup Kaggle details

In [33]:
# Kaggle json file and directory setup
os.environ['KAGGLE_CONFIG_DIR'] = os.getcwd()
! chmod 600 kaggle.json

Kaggle download settings and download

In [34]:
KAGGLE_DATASET_URL = 'jakeshbohaju/brain-tumor'
DESTINATION_FOLDER = 'input/'
! kaggle datasets download -d $KAGGLE_DATASET_URL -p $DESTINATION_FOLDER


100%|██████████████████████████████████████| 14.0M/14.0M [00:01<00:00, 18.2MB/s]
100%|██████████████████████████████████████| 14.0M/14.0M [00:01<00:00, 13.8MB/s]


Unzip the downloaded file, and delete the zip file.

Unzip the downloaded file, and delete the zip file.

In [35]:
import zipfile
with zipfile.ZipFile(DESTINATION_FOLDER + '/brain-tumor.zip' , 'r') as zip_ref:
    zip_ref.extractall(DESTINATION_FOLDER)

os.remove(DESTINATION_FOLDER + '/brain-tumor.zip')

Rename directories and files

In [36]:
! ls input/

'Brain Tumor'  'Brain Tumor.csv'   bt_dataset_t3.csv


In [37]:
! mv 'input/Brain Tumor.csv' input/brain-tumor.csv
! mv input/Brain\ Tumor/ input/brain-tumor/
! mv input/brain-tumor/Brain\ Tumor/ input/brain-tumor/brain-tumor/

---

# Data Preparation

---

## Data Cleaning

1. Sort the image files into tumor and non-tumor directories
2. Remove non image files
3. Remove empty directories

In [38]:
! ls input/brain-tumor

brain-tumor


In [39]:
# Change the dir structure of the input folder
! mkdir input/mri-brain-tumor/
! cp input/brain-tumor/brain-tumor/* input/mri-brain-tumor/
! rm -rf input/brain-tumor/ 

In [40]:
# classify images according to the target 'Class'
import pandas as pd
df = pd.read_csv('input/brain-tumor.csv')

# take out Image and Class only into a new data set
new_df = df[['Image', 'Class']]
new_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3762 entries, 0 to 3761
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Image   3762 non-null   object
 1   Class   3762 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 58.9+ KB


In [41]:
import shutil

# make new directories mri-tumor and mri-non-tumor in input
os.mkdir('input/mri-tumor/')
os.mkdir('input/mri-non-tumor/')

# move files according to the class
for index, row in new_df.iterrows():
    image_file = row['Image'] + '.jpg'
    image_class = row['Class']
    # save the image into the folder according to the class
    if image_class == 0:
        # save the image into the folder according to the class
        shutil.move('input/mri-brain-tumor/'+ image_file, 'input/mri-non-tumor/')
    else:
        # save the image into the folder according to the class
        shutil.move('input/mri-brain-tumor/'+ image_file, 'input/mri-tumor/')


In [49]:
# remove non image files and empty folders
! rm input/*.csv
! rm -rf input/mri-brain-tumor


In [50]:
os.listdir('input')


['mri-tumor', 'mri-non-tumor']

---

NOTE

* You may add as many sections as you want, as long as it supports your project workflow.
* All notebook's cells should be run top-down (you can't create a dynamic wherein a given point you need to go back to a previous cell to execute some task, like go back to a previous cell and refresh a variable content)

---

# Push files to Repo

* In case you don't need to push files to Repo, you may replace this section with "Conclusions and Next Steps" and state your conclusions and next steps.

In [ ]:
import os
try:
  # create here your folder
  # os.makedirs(name='')
except Exception as e:
  print(e)
